# The Task
Analyze user activity data from "dormat.csv" to identify dormant users and their activity periods. Calculate the average interval between activities, the time since the last activity, and a dormancy score for each user. Visualize the average interval and last played time for each user. Mark users as active or inactive for specific time periods based on their activity. Present the list of dormant users and their activity status over time.

In [3]:
import pandas as pd

df = pd.read_csv('dormat.csv')
display(df.head())

,userId,totalShots,createdAt
0,11686,186,2025-10-08 11:16:50.587
1,11686,16,2025-10-07 11:16:50.587
2,11686,97,2025-10-06 11:16:50.587
3,11686,65,2025-10-05 11:16:50.587
4,11686,8,2025-10-04 11:16:50.587


In [4]:
df = df.sort_values(by=['userId', 'createdAt'])
df['createdAt'] = pd.to_datetime(df['createdAt'])
df['time_interval'] = df.groupby('userId')['createdAt'].diff()


current_time = pd.to_datetime('2025-11-04 05:49:26')
last_activity_time = df.groupby('userId')['createdAt'].max().reset_index()
last_activity_time['time_since_last_activity'] = current_time - last_activity_time['createdAt']


df = pd.merge(df, last_activity_time[['userId', 'time_since_last_activity']], on='userId', how='left')

display(df.head())

,userId,totalShots,createdAt,time_interval,time_since_last_activity
0,112,7,2025-09-20 12:08:35.005,NaT,44 days 17:40:50.995000
1,116,174,2025-09-20 14:52:41.303,NaT,44 days 14:56:44.697000
2,126,4,2025-05-27 12:46:40.474,NaT,160 days 17:02:45.526000
3,148,4,2025-05-14 11:30:35.582,NaT,11 days 18:08:42.657000
4,148,18,2025-09-23 09:13:57.272,131 days 21:43:21.690000,11 days 18:08:42.657000


In [5]:
user_activity_stats = df.groupby('userId')['time_interval'].agg(['mean', 'std']).reset_index()
user_activity_stats = user_activity_stats.rename(columns={'mean': 'avg_time_interval', 'std': 'std_time_interval'})
display(user_activity_stats.head())

,userId,avg_time_interval,std_time_interval
0,112,NaT,NaT
1,116,NaT,NaT
2,126,NaT,NaT
3,148,6 days 05:32:41.836961538,26 days 06:13:55.867159172
4,621,13 days 13:55:03.036052631,30 days 00:06:23.239208598


In [6]:
merged_df = pd.merge(user_activity_stats, last_activity_time, on='userId', how='left')

merged_df['dormancy_threshold'] = merged_df['avg_time_interval'] + 2 * merged_df['std_time_interval']

display(merged_df.head())

,userId,avg_time_interval,std_time_interval,createdAt,time_since_last_activity,dormancy_threshold
0,112,NaT,NaT,2025-09-20 12:08:35.005,44 days 17:40:50.995000,NaT
1,116,NaT,NaT,2025-09-20 14:52:41.303,44 days 14:56:44.697000,NaT
2,126,NaT,NaT,2025-05-27 12:46:40.474,160 days 17:02:45.526000,NaT
3,148,6 days 05:32:41.836961538,26 days 06:13:55.867159172,2025-10-23 11:40:43.343,11 days 18:08:42.657000,58 days 18:00:33.571279882
4,621,13 days 13:55:03.036052631,30 days 00:06:23.239208598,2025-09-29 07:25:36.164,35 days 22:23:49.836000,73 days 14:07:49.514469827


## Calculate dormancy score

### Subtask:
For each user, calculate the dormancy score based on the time since their last activity and their dormancy threshold.


**Reasoning**:
Calculate the dormancy score based on the time since last activity and the dormancy threshold, handling potential division by zero or NaT values.



In [8]:
# Calculate dormancy score, handling potential division by zero or NaT
merged_df['dormancy_score'] = merged_df['time_since_last_activity'] / merged_df['dormancy_threshold']

# Handle cases where dormancy_threshold is NaT (e.g., user with only one activity)
# In these cases, the dormancy score will be NaT, which is a reasonable representation.
# Handle cases where dormancy_threshold is 0 (which is already handled by division by zero resulting in inf or NaN depending on numerator)
# If time_since_last_activity is also 0 and dormancy_threshold is 0, the result is NaN.

# Display the head of the merged_df
display(merged_df.head())

,userId,avg_time_interval,std_time_interval,createdAt,time_since_last_activity,dormancy_threshold,dormancy_score
0,112,NaT,NaT,2025-09-20 12:08:35.005,44 days 17:40:50.995000,NaT,NaN
1,116,NaT,NaT,2025-09-20 14:52:41.303,44 days 14:56:44.697000,NaT,NaN
2,126,NaT,NaT,2025-05-27 12:46:40.474,160 days 17:02:45.526000,NaT,NaN
3,148,6 days 05:32:41.836961538,26 days 06:13:55.867159172,2025-10-23 11:40:43.343,11 days 18:08:42.657000,58 days 18:00:33.571279882,0.200102
4,621,13 days 13:55:03.036052631,30 days 00:06:23.239208598,2025-09-29 07:25:36.164,35 days 22:23:49.836000,73 days 14:07:49.514469827,0.488298


## Identify dormant users

### Subtask:
Mark users as dormant if their dormancy score exceeds a certain value (e.g., 1).


**Reasoning**:
Define a dormancy threshold and create a new column 'is_dormant' in `merged_df`, populating it based on whether the 'dormancy_score' exceeds the threshold, and then display the head of the updated DataFrame.



In [9]:
# Define the dormancy threshold
dormancy_threshold_value = 1

# Create the 'is_dormant' column
# Users with NaN dormancy scores are not considered dormant by this metric
merged_df['is_dormant'] = (merged_df['dormancy_score'] > dormancy_threshold_value)

# Display the head of the merged_df
display(merged_df.head())

,userId,avg_time_interval,std_time_interval,createdAt,time_since_last_activity,dormancy_threshold,dormancy_score,is_dormant
0,112,NaT,NaT,2025-09-20 12:08:35.005,44 days 17:40:50.995000,NaT,NaN,False
1,116,NaT,NaT,2025-09-20 14:52:41.303,44 days 14:56:44.697000,NaT,NaN,False
2,126,NaT,NaT,2025-05-27 12:46:40.474,160 days 17:02:45.526000,NaT,NaN,False
3,148,6 days 05:32:41.836961538,26 days 06:13:55.867159172,2025-10-23 11:40:43.343,11 days 18:08:42.657000,58 days 18:00:33.571279882,0.200102,False
4,621,13 days 13:55:03.036052631,30 days 00:06:23.239208598,2025-09-29 07:25:36.164,35 days 22:23:49.836000,73 days 14:07:49.514469827,0.488298,False


## Analyze activity periods

### Subtask:
For each user, analyze their activity periods and mark them as active or inactive for specific time periods (e.g., month-wise).


**Reasoning**:
Extract the month and year from the `createdAt` column and then group by `userId` and the new 'activity_month' to count activities per month per user. Finally, pivot the table to show activity status by month for each user and display the head of the result.



In [10]:
df['activity_month'] = df['createdAt'].dt.to_period('M')

monthly_activity = df.groupby(['userId', 'activity_month']).size().reset_index(name='activity_count')

monthly_activity_status = monthly_activity.pivot_table(index='userId', columns='activity_month', values='activity_count', fill_value=0)
monthly_activity_status = monthly_activity_status.applymap(lambda x: 1 if x > 0 else 0)

display(monthly_activity_status.head())

/tmp/ipython-input-3070144834.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  monthly_activity_status = monthly_activity_status.applymap(lambda x: 1 if x > 0 else 0)


activity_month,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10
userId,,,,,,,,,,
112,0,0,0,0,0,0,0,0,1,0
116,0,0,0,0,0,0,0,0,1,0
126,0,0,0,0,1,0,0,0,0,0
148,0,0,0,0,1,0,0,0,1,1
621,1,1,0,0,0,1,0,0,1,0


## Visualize results

### Subtask:
Visualize the average interval and last played time for each user.


## Summary:

### Data Analysis Key Findings

* The average and standard deviation of time intervals between activities were calculated for each user.
* A dormancy threshold was determined for each user based on their average interval and standard deviation.
* A dormancy score was calculated for each user by dividing the time since their last activity by their dormancy threshold.
* Users were marked as dormant if their dormancy score exceeded 1.
* User activity periods were analyzed month-wise, indicating whether a user was active (1) or inactive (0) in a given month.
* A scatter plot was generated to visualize the relationship between the average time interval between activities and the time since the last activity for each user.

### Monthly Dormancy Analysis Rationale:

Analyzing user dormancy on a monthly basis provides a more granular view of user engagement over time. It allows us to:

* **Identify trends:** Observe how dormancy patterns evolve from month to month.
* **Pinpoint periods of inactivity:** Clearly see when users become inactive and for how long.
* **Assess the impact of interventions:** If any strategies are implemented to re-engage users, monthly analysis can help evaluate their effectiveness.
* **Tailor re-engagement efforts:** Understand the typical activity patterns within each month to inform targeted campaigns.

By examining dormancy month-by-month, we can gain deeper insights into user behavior and develop more effective strategies to retain users.

### Insights or Next Steps

* Iterate through each month and calculate the average interval, time since last activity, dormancy score, and status for each user within that month.
* Present the list of dormant users and their activity status over time in a clear, month-by-month format.
* Investigate users with `NaN` dormancy scores (likely due to having only one activity) to determine an appropriate approach for classifying their dormancy status.

In [14]:
import altair as alt

# Convert timedelta columns to days (float) for plotting
merged_df['avg_time_interval_days'] = merged_df['avg_time_interval'].dt.days
merged_df['time_since_last_activity_days'] = merged_df['time_since_last_activity'].dt.days

# Create a new DataFrame with only the necessary columns for plotting
plot_df = merged_df[['userId', 'avg_time_interval_days', 'time_since_last_activity_days', 'dormancy_score']].copy()

# Create the scatter plot using the new DataFrame
chart = alt.Chart(plot_df).mark_point().encode(
    x=alt.X('avg_time_interval_days', title='Average Time Interval Between Activities (Days)'),
    y=alt.Y('time_since_last_activity_days', title='Time Since Last Activity (Days)'),
    tooltip=['userId', 'avg_time_interval_days', 'time_since_last_activity_days', 'dormancy_score']
).properties(
    title='User Activity Analysis: Average Interval vs. Time Since Last Activity'
).interactive()

chart.display()

alt.Chart(...)

## Summary:

### Data Analysis Key Findings

*   The average and standard deviation of time intervals between activities were calculated for each user.
*   A dormancy threshold was determined for each user based on their average interval and standard deviation.
*   A dormancy score was calculated for each user by dividing the time since their last activity by their dormancy threshold.
*   Users were marked as dormant if their dormancy score exceeded 1.
*   User activity periods were analyzed month-wise, indicating whether a user was active (1) or inactive (0) in a given month.
*   A scatter plot was generated to visualize the relationship between the average time interval between activities and the time since the last activity for each user.

### Insights or Next Steps

*   Identify and list the users marked as dormant and present their monthly activity status to provide a clear view of their inactivity periods.
*   Investigate users with `NaN` dormancy scores (likely due to having only one activity) to determine an appropriate approach for classifying their dormancy status.


## Prepare data for modeling

### Subtask:
Engineer features from the user activity data that could be relevant for predicting future dormancy. This might include features like average interval, standard deviation of intervals, total activity count, recency of last activity, and the proportion of active months. Define the target variable for the model. Split the data into training and testing sets.

**Reasoning**:
Aggregate the data to create features for each user, such as the total number of activities, the average and standard deviation of time intervals, and the time since the last activity. Define the target variable as the dormancy status in the last month available in the data. Select the features and target, handle missing values, and split the data into training and testing sets.

In [33]:
from sklearn.model_selection import train_test_split
import numpy as np
# Aggregate data to create features for each user
user_features = df.groupby('userId').agg(
    total_activities=('createdAt', 'count'),
    avg_interval=('time_interval', lambda x: x.mean().total_seconds() if pd.notna(x.mean()) else 0),
    std_interval=('time_interval', lambda x: x.std().total_seconds() if pd.notna(x.std()) else 0),
    last_activity_time=('createdAt', 'max')
).reset_index()

# Calculate time since last activity (using the current_time defined earlier)
user_features['time_since_last_activity'] = (current_time - user_features['last_activity_time']).dt.total_seconds()

# Merge with dormancy status (using the 'is_dormant' from merged_df as the target)
user_features = pd.merge(user_features, merged_df[['userId', 'is_dormant']], on='userId', how='left')

# Define features (X) and target (y)
# Exclude 'userId', 'last_activity_time' and the target 'is_dormant'
X = user_features[['total_activities', 'avg_interval', 'std_interval', 'time_since_last_activity']]
y = user_features['is_dormant']

# Handle potential infinite values resulting from division by zero in avg_interval or std_interval
# (This might occur if a user has only one activity)
# Replace infinite values with a large number or the maximum finite value in the column
X.replace([np.inf, -np.inf], np.finfo(np.float64).max, inplace=True)

# Handle other potential NaN values (e.g., for users with only one activity where std_interval is NaN)
X.fillna(0, inplace=True)


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of training data (X_train, y_train):", X_train.shape, y_train.shape)
print("Shape of testing data (X_test, y_test):", X_test.shape, y_test.shape)

Shape of training data (X_train, y_train): (360, 4) (360,)
Shape of testing data (X_test, y_test): (90, 4) (90,)


/tmp/ipython-input-2704344789.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.replace([np.inf, -np.inf], np.finfo(np.float64).max, inplace=True)
/tmp/ipython-input-2704344789.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(0, inplace=True)


## Build an unsupervised model (KMeans Clustering)

### Subtask:
Train a KMeans clustering model on the user activity features and save the trained model as a pickle file.

**Reasoning**:
Import necessary libraries, choose KMeans for clustering, train the model on the user features (excluding the target variable), and save the trained model to a pickle file.

In [35]:
from sklearn.cluster import KMeans
import pickle

# Choose the number of clusters (you can adjust this based on your needs)
n_clusters = 3

# Choose an unsupervised model (KMeans Clustering)
unsupervised_model = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)

# Train the model on the user features (X)
# We use the same features X that were prepared for the supervised model
unsupervised_model.fit(X)

# Save the trained unsupervised model to a pickle file
filename = 'unsupervised_dormancy_model.pkl'
pickle.dump(unsupervised_model, open(filename, 'wb'))

print(f"Unsupervised model saved to {filename}")

Unsupervised model saved to unsupervised_dormancy_model.pkl


## Build a dormancy prediction model

### Subtask:
Choose an appropriate machine learning model for classification, train it, evaluate its performance, and save the trained model as a pickle file.

**Reasoning**:
Import necessary libraries, choose a suitable classification model (e.g., RandomForestClassifier), train the model on the training data, evaluate its performance using relevant metrics, and finally save the trained model to a pickle file.

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

# Choose a model (Random Forest Classifier is a good starting point)
model = RandomForestClassifier(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

# Save the trained model to a pickle file
filename = 'dormancy_model.pkl'
pickle.dump(model, open(filename, 'wb'))

print(f"Model saved to {filename}")

Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

       False       1.00      1.00      1.00        35
        True       1.00      1.00      1.00        55

    accuracy                           1.00        90
   macro avg       1.00      1.00      1.00        90
weighted avg       1.00      1.00      1.00        90

Model saved to dormancy_model.pkl


## Rationale for Monthly Analysis

We proceeded with a monthly analysis instead of a single overall dormancy classification as initially outlined in the document's Phase 1 because a monthly approach offers a more dynamic and granular understanding of user activity and dormancy.

While an overall dormancy score gives a general idea of a user's historical engagement, it doesn't capture fluctuations in activity over time. A user might have been very active historically but become dormant recently. A single dormancy score might still classify them as active or at risk based on their overall average, masking their current inactivity.

By analyzing dormancy month-by-month, we can:

*   **Identify Recent Dormancy:** Quickly pinpoint users who have become inactive in recent months, even if their historical activity was high.
*   **Track Activity Patterns:** Observe seasonal trends or responses to interventions by seeing how dormancy status changes each month.
*   **Tailor Engagement Strategies:** Develop more targeted re-engagement campaigns based on when users became dormant and their activity in specific months.
*   **Assess Impact of Events:** Analyze the impact of product updates, marketing campaigns, or other events on user activity and dormancy in specific months.

This monthly breakdown provides actionable insights that a single, overall dormancy score would not, allowing for more timely and effective interventions to retain users.

## Save Monthly Dormancy Data to CSV

### Subtask:
Save the formatted monthly dormancy data for each month to a separate CSV file.

**Reasoning**:
Iterate through the unique months in the `formatted_monthly_dormancy` DataFrame, filter the data for each month, and save the filtered data to a CSV file named with the corresponding month.

In [24]:
import os

# Create a directory to save the monthly CSVs
output_dir = "monthly_dormancy_data"
os.makedirs(output_dir, exist_ok=True)

# Iterate through unique months and save data to CSV
for month in all_monthly_dormancy_df['month'].unique():
    month_data_formatted = formatted_monthly_dormancy[all_monthly_dormancy_df['month'] == month].copy()
    # Format the month for the filename
    month_str = month.strftime('%Y-%m')
    output_filename = os.path.join(output_dir, f'dormancy_data_{month_str}.csv')
    month_data_formatted.to_csv(output_filename, index=False)
    print(f"Saved dormancy data for {month} to {output_filename}")

Saved dormancy data for 2025-09 to monthly_dormancy_data/dormancy_data_2025-09.csv
Saved dormancy data for 2025-05 to monthly_dormancy_data/dormancy_data_2025-05.csv
Saved dormancy data for 2025-10 to monthly_dormancy_data/dormancy_data_2025-10.csv
Saved dormancy data for 2025-01 to monthly_dormancy_data/dormancy_data_2025-01.csv
Saved dormancy data for 2025-02 to monthly_dormancy_data/dormancy_data_2025-02.csv
Saved dormancy data for 2025-06 to monthly_dormancy_data/dormancy_data_2025-06.csv
Saved dormancy data for 2025-03 to monthly_dormancy_data/dormancy_data_2025-03.csv
Saved dormancy data for 2025-04 to monthly_dormancy_data/dormancy_data_2025-04.csv
Saved dormancy data for 2025-07 to monthly_dormancy_data/dormancy_data_2025-07.csv
Saved dormancy data for 2025-08 to monthly_dormancy_data/dormancy_data_2025-08.csv


## Visualize Monthly Dormancy Proportion

### Subtask:
Generate a bar chart showing the proportion of users in each dormancy category (ACTIVE, AT_RISK, DORMANT, UNKNOWN) for each month.

**Reasoning**:
Group the `all_monthly_dormancy_df` by month and monthly status, count the occurrences of each status, calculate the proportion of each status within each month, and then create a grouped bar chart using Altair to visualize these proportions across months.

In [26]:
import altair as alt

# Calculate the counts of each status per month
monthly_status_counts = all_monthly_dormancy_df.groupby(['month', 'monthly_status']).size().reset_index(name='count')

# Calculate the total number of users per month
monthly_total_users = all_monthly_dormancy_df.groupby('month').size().reset_index(name='total_users')

# Merge to calculate proportions
monthly_status_proportions = pd.merge(monthly_status_counts, monthly_total_users, on='month')
monthly_status_proportions['proportion'] = monthly_status_proportions['count'] / monthly_status_proportions['total_users']

# Convert month to string for plotting and drop the original 'month' column
monthly_status_proportions['month_str'] = monthly_status_proportions['month'].astype(str)
monthly_status_proportions = monthly_status_proportions.drop(columns=['month'])


# Create the grouped bar chart
chart = alt.Chart(monthly_status_proportions).mark_bar().encode(
    x=alt.X('month_str:N', axis=alt.Axis(title='Month')),
    y=alt.Y('proportion:Q', axis=alt.Axis(title='Proportion of Users')),
    color='monthly_status:N',
    tooltip=['month_str', 'monthly_status', 'proportion', 'count']
).properties(
    title='Proportion of User Dormancy Status by Month'
).interactive()

chart.display()

alt.Chart(...)

## Identify and list dormant users and their monthly activity status

### Subtask:
Filter the `merged_df` to get the dormant users and then merge this with the `monthly_activity_status` DataFrame to show their activity status for each month.

**Reasoning**:
Filter the `merged_df` DataFrame to include only the rows where 'is_dormant' is True, and then merge the resulting DataFrame with the `monthly_activity_status` DataFrame using the 'userId' column to combine the dormancy information with the monthly activity status. Finally, display the head of the merged DataFrame.

In [15]:
# Filter for dormant users
dormant_users_df = merged_df[merged_df['is_dormant']].copy()

# Merge with monthly activity status
dormant_users_monthly_status = pd.merge(dormant_users_df[['userId', 'dormancy_score']],
                                         monthly_activity_status,
                                         on='userId',
                                         how='left')

display(dormant_users_monthly_status.head())

,userId,dormancy_score,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10
0,642,6.783965,1,1,1,1,1,0,0,0,0,0
1,643,2.269308,0,1,0,0,1,1,0,0,0,0
2,672,2.719757,1,1,1,1,1,1,1,1,1,1
3,809,24.883557,1,0,0,0,0,0,0,0,0,0
4,1789,3.153684,0,0,1,1,1,0,0,1,1,0


# Task
Analyze user dormancy status on a monthly basis using the provided data in "/content/The Dormancy Problem.docx" and "/content/dormat.csv". For each month, calculate the average activity interval, days since last activity, and a dormancy score for each user. Classify users into 'DORMANT', 'ACTIVE', or 'AT_RISK' based on their monthly dormancy score and a defined threshold. Output the results for each month in a CSV format with the columns 'user_id', 'avg_interval (days)', 'last_active_days_ago', 'dormancy_score', and 'status'. Additionally, identify and address users with NaN dormancy scores.

## Prepare data for monthly analysis

### Subtask:
Ensure the data is sorted and has the necessary time-based columns (e.g., 'activity_month').


**Reasoning**:
Sort the DataFrame by 'userId' and 'createdAt', convert 'createdAt' to datetime objects, and extract the month and year into an 'activity_month' column to prepare the data for monthly analysis.



In [16]:
df = df.sort_values(by=['userId', 'createdAt'])
df['createdAt'] = pd.to_datetime(df['createdAt'])
df['activity_month'] = df['createdAt'].dt.to_period('M')
display(df.head())

,userId,totalShots,createdAt,time_interval,time_since_last_activity,activity_month
0,112,7,2025-09-20 12:08:35.005,NaT,44 days 17:40:50.995000,2025-09
1,116,174,2025-09-20 14:52:41.303,NaT,44 days 14:56:44.697000,2025-09
2,126,4,2025-05-27 12:46:40.474,NaT,160 days 17:02:45.526000,2025-05
3,148,4,2025-05-14 11:30:35.582,NaT,11 days 18:08:42.657000,2025-05
4,148,18,2025-09-23 09:13:57.272,131 days 21:43:21.690000,11 days 18:08:42.657000,2025-09


## Iterate through months

### Subtask:
Iterate through each unique month present in the data.


**Reasoning**:
Extract the unique months from the 'activity_month' column and iterate through each unique month.



In [17]:
unique_months = df['activity_month'].unique()

for month in unique_months:
    print(f"Analyzing month: {month}")
    # The next steps for analyzing each month will be implemented in subsequent subtasks.


Analyzing month: 2025-09
Analyzing month: 2025-05
Analyzing month: 2025-10
Analyzing month: 2025-01
Analyzing month: 2025-02
Analyzing month: 2025-06
Analyzing month: 2025-03
Analyzing month: 2025-04
Analyzing month: 2025-07
Analyzing month: 2025-08


## Calculate monthly metrics

### Subtask:
For each month, calculate the average interval, time since last activity, and dormancy score for each user active in that month.


In [19]:
monthly_dormancy_data = []

for month in unique_months:
    print(f"Analyzing month: {month}")

    # Filter data for the current month
    month_data = df[df['activity_month'] == month].copy()

    if not month_data.empty:
        # Calculate average time interval and last activity time within the month
        monthly_stats = month_data.groupby('userId').agg(
            last_activity_month=('createdAt', 'max'),  # Corrected agg function
            avg_time_interval_month=('time_interval', 'mean')
        ).reset_index()

        # Calculate time since last activity for the current month
        end_of_month = (month.to_timestamp() + pd.offsets.MonthEnd(0))
        monthly_stats['last_active_days_ago'] = (end_of_month - monthly_stats['last_activity_month']).dt.days

        # Merge with overall dormancy threshold
        monthly_stats = pd.merge(monthly_stats, merged_df[['userId', 'dormancy_threshold']], on='userId', how='left')

        # Calculate monthly dormancy score
        # Convert timedelta to seconds for division, handle potential NaT or zero threshold
        monthly_stats['dormancy_threshold_seconds'] = monthly_stats['dormancy_threshold'].dt.total_seconds()
        monthly_stats['last_active_seconds_ago'] = (end_of_month - monthly_stats['last_activity_month']).dt.total_seconds()


        # Calculate dormancy score, handling division by zero or NaT
        monthly_stats['dormancy_score'] = monthly_stats.apply(
            lambda row: row['last_active_seconds_ago'] / row['dormancy_threshold_seconds']
            if pd.notna(row['last_active_seconds_ago']) and pd.notna(row['dormancy_threshold_seconds']) and row['dormancy_threshold_seconds'] != 0
            else (float('inf') if pd.notna(row['last_active_seconds_ago']) and (pd.isna(row['dormancy_threshold_seconds']) or row['dormancy_threshold_seconds'] == 0) else float('nan')),
            axis=1
        )

        # Add month information
        monthly_stats['month'] = month

        # Append to the list
        monthly_dormancy_data.append(monthly_stats)

# Concatenate all monthly dataframes
if monthly_dormancy_data:
    all_monthly_dormancy_df = pd.concat(monthly_dormancy_data, ignore_index=True)
    display(all_monthly_dormancy_df.head())
else:
    print("No monthly data to process.")

Analyzing month: 2025-09
Analyzing month: 2025-05
Analyzing month: 2025-10
Analyzing month: 2025-01
Analyzing month: 2025-02
Analyzing month: 2025-06
Analyzing month: 2025-03
Analyzing month: 2025-04
Analyzing month: 2025-07
Analyzing month: 2025-08


,userId,last_activity_month,avg_time_interval_month,last_active_days_ago,dormancy_threshold,dormancy_threshold_seconds,last_active_seconds_ago,dormancy_score,month
0,112,2025-09-20 12:08:35.005,NaT,9,NaT,NaN,820284.995,inf,2025-09
1,116,2025-09-20 14:52:41.303,NaT,9,NaT,NaN,810438.697,inf,2025-09
2,148,2025-09-24 08:13:57.864,5 days 12:51:48.428416666,5,58 days 18:00:33.571279882,5.076034e+06,488762.136,0.096288,2025-09
3,621,2025-09-29 07:25:36.164,29 days 00:09:16.075250,0,73 days 14:07:49.514469827,6.358070e+06,59663.836,0.009384,2025-09
4,672,2025-09-08 12:12:33.310,0 days 20:04:27.720705882,21,12 days 10:42:41.504496966,1.075362e+06,1856846.690,1.726719,2025-09


## Determine monthly status

### Subtask:
Based on the monthly dormancy score and a defined threshold, determine the dormancy status (DORMANT, ACTIVE, AT_RISK) for each user in that month.


**Reasoning**:
Define the dormancy status thresholds and create a new column 'monthly_status' based on the 'dormancy_score' and the defined thresholds.



In [20]:
# Define dormancy thresholds
dormant_threshold = 1.0
at_risk_threshold = 0.5

# Define a function to determine the monthly status
def determine_monthly_status(score):
    if pd.isna(score):
        return 'UNKNOWN' # Handle users with NaN scores, e.g., single activity
    elif score > dormant_threshold:
        return 'DORMANT'
    elif score > at_risk_threshold:
        return 'AT_RISK'
    else:
        return 'ACTIVE'

# Apply the function to create the 'monthly_status' column
all_monthly_dormancy_df['monthly_status'] = all_monthly_dormancy_df['dormancy_score'].apply(determine_monthly_status)

# Display the head of the DataFrame
display(all_monthly_dormancy_df.head())

,userId,last_activity_month,avg_time_interval_month,last_active_days_ago,dormancy_threshold,dormancy_threshold_seconds,last_active_seconds_ago,dormancy_score,month,monthly_status
0,112,2025-09-20 12:08:35.005,NaT,9,NaT,NaN,820284.995,inf,2025-09,DORMANT
1,116,2025-09-20 14:52:41.303,NaT,9,NaT,NaN,810438.697,inf,2025-09,DORMANT
2,148,2025-09-24 08:13:57.864,5 days 12:51:48.428416666,5,58 days 18:00:33.571279882,5.076034e+06,488762.136,0.096288,2025-09,ACTIVE
3,621,2025-09-29 07:25:36.164,29 days 00:09:16.075250,0,73 days 14:07:49.514469827,6.358070e+06,59663.836,0.009384,2025-09,ACTIVE
4,672,2025-09-08 12:12:33.310,0 days 20:04:27.720705882,21,12 days 10:42:41.504496966,1.075362e+06,1856846.690,1.726719,2025-09,DORMANT


## Format and present output

### Subtask:
Structure the monthly dormancy data into the requested format and present it, possibly by saving each month's data to a separate CSV file or displaying it month by month.


**Reasoning**:
Select and reorder the columns, rename them, convert the average interval to days, and then iterate through the unique months to display the data for each month.



In [21]:
# Select and reorder columns and rename them
formatted_monthly_dormancy = all_monthly_dormancy_df[[
    'userId',
    'avg_time_interval_month',
    'last_active_days_ago',
    'dormancy_score',
    'monthly_status'
]].copy()

formatted_monthly_dormancy = formatted_monthly_dormancy.rename(columns={
    'userId': 'user_id',
    'avg_time_interval_month': 'avg_interval (days)',
    'last_active_days_ago': 'last_active_days_ago',
    'dormancy_score': 'dormancy_score',
    'monthly_status': 'status'
})

# Convert 'avg_interval (days)' timedelta to days (numeric), handling NaT values
formatted_monthly_dormancy['avg_interval (days)'] = formatted_monthly_dormancy['avg_interval (days)'].dt.days

# Iterate through unique months and display the data for each month
unique_months_formatted_df = formatted_monthly_dormancy['avg_interval (days)'].unique()

for month in all_monthly_dormancy_df['month'].unique():
    print(f"Monthly Dormancy Data for {month}:")
    month_data_formatted = formatted_monthly_dormancy[all_monthly_dormancy_df['month'] == month]
    display(month_data_formatted)

Monthly Dormancy Data for 2025-09:


,user_id,avg_interval (days),last_active_days_ago,dormancy_score,status
0,112,NaN,9,inf,DORMANT
1,116,NaN,9,inf,DORMANT
2,148,5.0,5,0.096288,ACTIVE
3,621,29.0,0,0.009384,ACTIVE
4,672,0.0,21,1.726719,DORMANT
5,1789,1.0,10,0.725179,AT_RISK
6,2357,2.0,21,1.827560,DORMANT
7,3411,0.0,20,0.902121,AT_RISK
8,3532,0.0,26,1.297013,DORMANT
9,4437,1.0,22,3.985952,DORMANT


Monthly Dormancy Data for 2025-05:


,user_id,avg_interval (days),last_active_days_ago,dormancy_score,status
21,126,NaN,3,inf,DORMANT
22,148,NaN,16,0.281197,ACTIVE
23,641,0.0,2,0.082739,ACTIVE
24,642,34.0,17,0.679550,AT_RISK
25,643,92.0,2,0.044109,ACTIVE
26,672,2.0,2,0.201887,ACTIVE
27,1025,2.0,28,0.376474,ACTIVE
28,1789,0.0,4,0.310086,ACTIVE
29,2357,0.0,2,0.202286,ACTIVE
30,3195,11.0,4,0.115760,ACTIVE


Monthly Dormancy Data for 2025-10:


...

,user_id,avg_interval (days),last_active_days_ago,dormancy_score,status
488,641,1.0,4,0.154250,ACTIVE
489,642,2.0,20,0.804633,AT_RISK
490,672,1.0,6,0.525971,AT_RISK
491,1025,1.0,1,0.018892,ACTIVE
492,1789,1.0,6,0.457151,ACTIVE
493,2357,2.0,14,1.229095,DORMANT
494,3195,NaN,19,0.496764,ACTIVE
495,3532,1.0,-1,-0.018360,ACTIVE
496,4571,4.0,18,1.243688,DORMANT
497,6886,0.0,11,0.147959,ACTIVE


Monthly Dormancy Data for 2025-07:


,user_id,avg_interval (days),last_active_days_ago,dormancy_score,status
524,672,0.0,14,1.183071,DORMANT
525,2357,3.0,-1,-0.067235,ACTIVE
526,3411,13.0,-1,-0.021647,ACTIVE
527,11408,3.0,-1,-0.056013,ACTIVE
528,11410,NaN,28,inf,DORMANT
529,11411,9.0,26,1.252104,DORMANT
530,11543,NaN,29,inf,DORMANT
531,11548,NaN,28,inf,DORMANT
532,11549,NaN,28,inf,DORMANT
533,11550,0.0,28,inf,DORMANT


Monthly Dormancy Data for 2025-08:


,user_id,avg_interval (days),last_active_days_ago,dormancy_score,status
536,672,2.0,5,4.588205e-01,ACTIVE
537,1789,1.0,2,1.689267e-01,ACTIVE
538,2357,2.0,8,7.091658e-01,AT_RISK
539,3411,0.0,1,6.488374e-02,ACTIVE
540,3532,93.0,1,7.812364e-02,ACTIVE
541,4437,0.0,8,1.537819e+00,DORMANT
542,6886,40.0,11,1.510058e-01,ACTIVE
543,11408,0.0,1,3.339287e-01,ACTIVE
544,11411,14.0,-1,-9.570769e-03,ACTIVE
545,11485,0.0,16,8.970110e-01,AT_RISK


## Handle nan dormancy scores

### Subtask:
Address users with NaN dormancy scores (likely due to single activity) and decide how to classify their status in the monthly analysis.


In [23]:
# Identify users with NaN dormancy scores in the monthly dormancy DataFrame
initial_nan_dormancy_mask = all_monthly_dormancy_df['dormancy_score'].isna()
nan_dormancy_users = all_monthly_dormancy_df[initial_nan_dormancy_mask].copy()


print("Users with NaN monthly dormancy scores:")
display(nan_dormancy_users.head())

# Analyze characteristics of users with NaN dormancy scores
# These users likely have only one activity in that specific month, resulting in a NaT time_interval,
# which propagates to NaN average time interval and thus NaN dormancy score when threshold is NaT.
# Let's count how many activities these users had in the months where their score is NaN.
# Corrected column name from 'user_id' to 'userId'
nan_dormancy_activity_counts = df[df.apply(lambda row: (row['userId'], row['activity_month']) in zip(nan_dormancy_users['userId'], nan_dormancy_users['month']), axis=1)].groupby(['userId', 'activity_month']).size().reset_index(name='activity_count_in_month')

print("\nActivity counts in months with NaN dormancy score:")
display(nan_dormancy_activity_counts.head())

# Based on the analysis (likely single activity in that month), a reasonable approach
# is to classify users with NaN dormancy scores based on the time since their last activity in that month.
# If the time since their last activity in that month is greater than a certain threshold (e.g., 30 days),
# they could be considered 'DORMANT' for that month. Otherwise, they could be 'AT_RISK' or 'ACTIVE'
# depending on the time frame and their overall activity patterns.
# However, given the prompt asks to address NaN dormancy scores which are present due to NaN threshold from overall analysis
# (mostly from single activity users overall), classifying them as 'UNKNOWN' when the score is NaN seems appropriate
# as their regular activity pattern (and thus dormancy threshold) couldn't be established.
# If the NaN is due to the monthly calculation (single activity in that month), we can use the overall dormancy score if available.

# Let's refine the approach: If monthly dormancy score is NaN, we will check the overall dormancy score.
# If overall dormancy score is also NaN (single activity overall), classify as 'UNKNOWN'.
# If overall dormancy score is available, use the overall dormancy status.
# This requires merging with the merged_df which has the overall dormancy status.

# Merge all_monthly_dormancy_df with merged_df to get overall dormancy status
all_monthly_dormancy_df = pd.merge(all_monthly_dormancy_df, merged_df[['userId', 'is_dormant']], on='userId', how='left')

# Update the 'monthly_status' for users with NaN dormancy scores
def update_monthly_status_for_nan(row):
    if pd.isna(row['dormancy_score']):
        if pd.isna(row['is_dormant']):
            return 'UNKNOWN' # Single activity overall and in the month
        elif row['is_dormant']:
            return 'DORMANT' # Single activity in the month, but dormant overall
        else:
            # Single activity in the month, but active overall. This case is less likely
            # to have a NaN monthly dormancy score unless the single activity in the month
            # is the only activity overall and is recent. We can consider them 'AT_RISK'
            # or 'ACTIVE' based on the time since that single activity relative to a general threshold.
            # Let's use 'AT_RISK' for now to highlight these users.
             return 'AT_RISK' # Changed from 'UNKNOWN' as per the previous logic in determine_monthly_status
    else:
        return row['monthly_status']

all_monthly_dormancy_df['monthly_status'] = all_monthly_dormancy_df.apply(update_monthly_status_for_nan, axis=1)

# Display the updated statuses for users who initially had NaN dormancy scores
# Corrected column name from 'user_id' to 'userId'
updated_nan_dormancy_users = all_monthly_dormancy_df[initial_nan_dormancy_mask] # Use the mask from before the update
print("\nUpdated monthly status for users who initially had NaN dormancy scores:")
display(updated_nan_dormancy_users[['userId', 'dormancy_score', 'monthly_status', 'is_dormant']].head())

# Remove the temporary 'is_dormant' column if not needed for future steps
all_monthly_dormancy_df = all_monthly_dormancy_df.drop(columns=['is_dormant'])

Users with NaN monthly dormancy scores:


,userId,last_activity_month,avg_time_interval_month,last_active_days_ago,dormancy_threshold,dormancy_threshold_seconds,last_active_seconds_ago,dormancy_score,month,monthly_status



Activity counts in months with NaN dormancy score:


,userId,activity_month,activity_count_in_month



Updated monthly status for users who initially had NaN dormancy scores:


,userId,dormancy_score,monthly_status,is_dormant


## Summary:

### Data Analysis Key Findings

*   The data was successfully processed to include an `activity_month` column, enabling monthly analysis.
*   Monthly metrics, including the average activity interval, days since last activity, and dormancy score, were calculated for each user active in a given month.
*   Users were classified into 'DORMANT', 'ACTIVE', or 'AT\_RISK' categories based on their monthly dormancy scores and defined thresholds (score > 1.0 for DORMANT, score > 0.5 for AT\_RISK, otherwise ACTIVE).
*   The analysis confirmed that there were no users with NaN dormancy scores in the monthly analysis data, making the specific handling of this case unnecessary for this dataset.
*   The final output includes the calculated metrics and status for each user on a monthly basis, formatted with columns 'user\_id', 'avg\_interval (days)', 'last\_active\_days\_ago', 'dormancy\_score', and 'status'.

### Insights or Next Steps

*   The monthly dormancy classification provides a granular view of user engagement fluctuations over time, which can inform targeted monthly retention campaigns.
*   Saving the monthly dormancy data to separate CSV files for each month would facilitate easier sharing and further analysis or reporting.


# Task
Analyze user dormancy status transitions over time, visualize these transitions, build a machine learning model to predict user dormancy, and explain the model and its results using data from "/content/The Dormancy Problem.docx" and "/content/dormat.csv".

## Analyze dormancy status transitions

### Subtask:
Prepare the monthly dormancy data to track user status changes between consecutive months.


**Reasoning**:
Sort the DataFrame, create the previous month status column using shift within user groups, fill NaN values, and display the head of the DataFrame.



In [27]:
all_monthly_dormancy_df = all_monthly_dormancy_df.sort_values(by=['userId', 'month'])
all_monthly_dormancy_df['previous_month_status'] = all_monthly_dormancy_df.groupby('userId')['monthly_status'].shift(1)
all_monthly_dormancy_df['previous_month_status'] = all_monthly_dormancy_df['previous_month_status'].fillna('NO_PREVIOUS_ACTIVITY')

display(all_monthly_dormancy_df.head())

,userId,last_activity_month,avg_time_interval_month,last_active_days_ago,dormancy_threshold,dormancy_threshold_seconds,last_active_seconds_ago,dormancy_score,month,monthly_status,previous_month_status
0,112,2025-09-20 12:08:35.005,NaT,9,NaT,NaN,820284.995,inf,2025-09,DORMANT,NO_PREVIOUS_ACTIVITY
1,116,2025-09-20 14:52:41.303,NaT,9,NaT,NaN,810438.697,inf,2025-09,DORMANT,NO_PREVIOUS_ACTIVITY
21,126,2025-05-27 12:46:40.474,NaT,3,NaT,NaN,299599.526,inf,2025-05,DORMANT,NO_PREVIOUS_ACTIVITY
22,148,2025-05-14 11:30:35.582,NaT,16,58 days 18:00:33.571279882,5.076034e+06,1427364.418,0.281197,2025-05,ACTIVE,NO_PREVIOUS_ACTIVITY
2,148,2025-09-24 08:13:57.864,5 days 12:51:48.428416666,5,58 days 18:00:33.571279882,5.076034e+06,488762.136,0.096288,2025-09,ACTIVE,ACTIVE


## Analyze dormancy status transitions

### Subtask:
Calculate the number of users transitioning between different dormancy statuses (e.g., DORMANT to ACTIVE, ACTIVE to DORMANT) for each month.


**Reasoning**:
Create a new DataFrame by grouping `all_monthly_dormancy_df` by 'month', 'previous_month_status', and 'monthly_status', and then counting the occurrences of each transition using `.size()`. Reset the index of the resulting DataFrame. Rename the column containing the counts to 'transition_count', and display the head of the DataFrame.



In [28]:
monthly_transitions = all_monthly_dormancy_df.groupby(['month', 'previous_month_status', 'monthly_status']).size().reset_index(name='transition_count')
display(monthly_transitions.head())

,month,previous_month_status,monthly_status,transition_count
0,2025-01,NO_PREVIOUS_ACTIVITY,ACTIVE,45
1,2025-01,NO_PREVIOUS_ACTIVITY,AT_RISK,17
2,2025-01,NO_PREVIOUS_ACTIVITY,DORMANT,209
3,2025-02,ACTIVE,ACTIVE,8
4,2025-02,ACTIVE,DORMANT,9


## Analyze dormancy status transitions

### Subtask:
Visualize the dormancy status transitions over time.


**Reasoning**:
Create a grouped bar chart to visualize the monthly dormancy status transitions using Altair.



In [29]:
import altair as alt

chart = alt.Chart(monthly_transitions).mark_bar().encode(
    x=alt.X('month:N', axis=alt.Axis(title='Month')),
    y=alt.Y('transition_count:Q', axis=alt.Axis(title='Number of Users')),
    color='monthly_status:N',
    column=alt.Column('previous_month_status:N', title='Previous Month Status'),
    tooltip=['month', 'previous_month_status', 'monthly_status', 'transition_count']
).properties(
    title='Monthly Dormancy Status Transitions'
).interactive()

chart.display()

TypeError: Object of type Period is not JSON serializable

alt.Chart(...)

**Reasoning**:
The error indicates that Altair cannot serialize the Period objects in the 'month' column for plotting. I need to convert the 'month' column in the `monthly_transitions` DataFrame to a string format before plotting.



In [30]:
import altair as alt

# Convert the 'month' column to string format for plotting
monthly_transitions['month_str'] = monthly_transitions['month'].astype(str)

chart = alt.Chart(monthly_transitions).mark_bar().encode(
    x=alt.X('month_str:N', axis=alt.Axis(title='Month')),
    y=alt.Y('transition_count:Q', axis=alt.Axis(title='Number of Users')),
    color='monthly_status:N',
    column=alt.Column('previous_month_status:N', title='Previous Month Status'),
    tooltip=['month_str', 'previous_month_status', 'monthly_status', 'transition_count']
).properties(
    title='Monthly Dormancy Status Transitions'
).interactive()

chart.display()

TypeError: Object of type Period is not JSON serializable

alt.Chart(...)

**Reasoning**:
The error indicates that even though I created a new string column for the month, Altair might still be referencing the original 'month' column which is of type Period. To ensure that only the string representation is used for plotting and avoid any potential issues with the original Period objects, I will create a new DataFrame containing only the necessary columns for plotting, including the string version of the month.



In [31]:
import altair as alt

# Create a new DataFrame with only the necessary columns for plotting
# Ensure 'month_str' is used and not the original 'month'
plot_transitions_df = monthly_transitions[['month_str', 'previous_month_status', 'monthly_status', 'transition_count']].copy()

chart = alt.Chart(plot_transitions_df).mark_bar().encode(
    x=alt.X('month_str:N', axis=alt.Axis(title='Month')),
    y=alt.Y('transition_count:Q', axis=alt.Axis(title='Number of Users')),
    color='monthly_status:N',
    column=alt.Column('previous_month_status:N', title='Previous Month Status'),
    tooltip=['month_str', 'previous_month_status', 'monthly_status', 'transition_count']
).properties(
    title='Monthly Dormancy Status Transitions'
).interactive()

chart.display()

alt.Chart(...)